# Exploratory Data Analysis: Brent Oil Prices

## Task 1: Laying the Foundation for Analysis

This notebook performs exploratory data analysis on historical Brent oil prices to:
1. Understand data structure and quality
2. Identify trends, seasonality, and volatility patterns
3. Test for stationarity
4. Visualize major events and their potential impact

**Author:** Daniel Mituku  
**Date:** February 2026

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Add src to path
import sys
sys.path.append('..')
from src.data_loader import load_brent_oil_data, load_events_data, add_derived_features, get_data_summary

## 1. Load and Inspect Data

In [ ]:
# Load Brent oil price data
# Note: Update the path to your actual data file
df = load_brent_oil_data('../data/raw/brent_oil_prices.csv')

# Display basic info
print("Data Shape:", df.shape)
print("\nData Types:")
print(df.dtypes)
print("\nFirst 5 rows:")
df.head()

In [ ]:
# Get summary statistics
summary = get_data_summary(df)
print("Data Summary:")
for key, value in summary.items():
    print(f"  {key}: {value}")

In [ ]:
# Add derived features
df = add_derived_features(df)
df.head()

## 2. Load Events Data

In [ ]:
# Load major events
events_df = load_events_data('../events/major_events.csv')
print(f"Loaded {len(events_df)} major events")
events_df

## 3. Visualize Price Series

In [ ]:
# Plot raw price series
fig, ax = plt.subplots(figsize=(16, 8))

ax.plot(df['Date'], df['Price'], 'b-', alpha=0.8, linewidth=0.8)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Price (USD/barrel)', fontsize=12)
ax.set_title('Brent Crude Oil Prices (1987-2022)', fontsize=14)

# Add event markers
colors = {'Conflict': 'red', 'OPEC Policy': 'green', 'Economic Crisis': 'orange', 'Sanctions': 'purple', 'Geopolitical': 'brown'}
for _, event in events_df.iterrows():
    if event['date'] >= df['Date'].min() and event['date'] <= df['Date'].max():
        color = colors.get(event['event_type'], 'gray')
        ax.axvline(event['date'], color=color, linestyle='--', alpha=0.5, linewidth=1)

plt.tight_layout()
plt.show()

In [ ]:
# Price distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(df['Price'].dropna(), bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Price (USD/barrel)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Brent Oil Prices')

# Box plot by decade
df['decade'] = (df['year'] // 10) * 10
df.boxplot(column='Price', by='decade', ax=axes[1])
axes[1].set_xlabel('Decade')
axes[1].set_ylabel('Price (USD/barrel)')
axes[1].set_title('Price Distribution by Decade')
plt.suptitle('')

plt.tight_layout()
plt.show()

## 4. Trend Analysis

In [ ]:
# Yearly average prices
yearly_avg = df.groupby('year')['Price'].mean()

fig, ax = plt.subplots(figsize=(14, 6))
ax.bar(yearly_avg.index, yearly_avg.values, color='steelblue', alpha=0.8)
ax.set_xlabel('Year')
ax.set_ylabel('Average Price (USD/barrel)')
ax.set_title('Average Annual Brent Oil Prices')
ax.axhline(yearly_avg.mean(), color='red', linestyle='--', label=f'Overall Mean: ${yearly_avg.mean():.2f}')
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Rolling statistics
fig, axes = plt.subplots(2, 1, figsize=(16, 10), sharex=True)

# Price with rolling mean
axes[0].plot(df['Date'], df['Price'], 'b-', alpha=0.5, label='Daily Price')
axes[0].plot(df['Date'], df['rolling_mean_30'], 'r-', linewidth=2, label='30-day Rolling Mean')
axes[0].set_ylabel('Price (USD/barrel)')
axes[0].set_title('Brent Oil Price with Rolling Mean')
axes[0].legend()

# Rolling volatility
axes[1].plot(df['Date'], df['rolling_volatility_30'], 'purple', linewidth=1)
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Annualized Volatility')
axes[1].set_title('30-day Rolling Volatility')

plt.tight_layout()
plt.show()

## 5. Log Returns Analysis

In [ ]:
# Plot log returns
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Time series of log returns
axes[0, 0].plot(df['Date'], df['log_return'], 'b-', alpha=0.6)
axes[0, 0].axhline(0, color='red', linestyle='--')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Log Return')
axes[0, 0].set_title('Daily Log Returns')

# Distribution of log returns
axes[0, 1].hist(df['log_return'].dropna(), bins=100, edgecolor='black', alpha=0.7, density=True)
axes[0, 1].set_xlabel('Log Return')
axes[0, 1].set_ylabel('Density')
axes[0, 1].set_title('Distribution of Log Returns')

# Q-Q plot
from scipy import stats
stats.probplot(df['log_return'].dropna(), dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot (Log Returns vs Normal)')

# ACF plot
plot_acf(df['log_return'].dropna(), lags=50, ax=axes[1, 1])
axes[1, 1].set_title('Autocorrelation of Log Returns')

plt.tight_layout()
plt.show()

In [ ]:
# Log returns statistics
log_returns = df['log_return'].dropna()
print("Log Returns Statistics:")
print(f"  Mean: {log_returns.mean():.6f}")
print(f"  Std Dev: {log_returns.std():.6f}")
print(f"  Skewness: {log_returns.skew():.4f}")
print(f"  Kurtosis: {log_returns.kurtosis():.4f}")
print(f"  Min: {log_returns.min():.6f}")
print(f"  Max: {log_returns.max():.6f}")

## 6. Stationarity Testing

In [ ]:
def test_stationarity(series, name):
    """Perform ADF and KPSS tests for stationarity."""
    print(f"\n{'='*60}")
    print(f"Stationarity Tests for: {name}")
    print('='*60)
    
    # ADF Test (Null: Non-stationary)
    adf_result = adfuller(series.dropna(), autolag='AIC')
    print(f"\nADF Test:")
    print(f"  Test Statistic: {adf_result[0]:.4f}")
    print(f"  p-value: {adf_result[1]:.4f}")
    print(f"  Critical Values:")
    for key, value in adf_result[4].items():
        print(f"    {key}: {value:.4f}")
    print(f"  Conclusion: {'Stationary' if adf_result[1] < 0.05 else 'Non-stationary'} (at 5% significance)")
    
    # KPSS Test (Null: Stationary)
    kpss_result = kpss(series.dropna(), regression='c', nlags='auto')
    print(f"\nKPSS Test:")
    print(f"  Test Statistic: {kpss_result[0]:.4f}")
    print(f"  p-value: {kpss_result[1]:.4f}")
    print(f"  Critical Values:")
    for key, value in kpss_result[3].items():
        print(f"    {key}: {value:.4f}")
    print(f"  Conclusion: {'Stationary' if kpss_result[1] > 0.05 else 'Non-stationary'} (at 5% significance)")

In [ ]:
# Test raw prices
test_stationarity(df['Price'], 'Raw Prices')

In [ ]:
# Test log returns
test_stationarity(df['log_return'], 'Log Returns')

## 7. Volatility Patterns

In [ ]:
# Volatility clustering visualization
fig, axes = plt.subplots(2, 1, figsize=(16, 10), sharex=True)

# Absolute returns (proxy for volatility)
axes[0].plot(df['Date'], df['log_return'].abs(), 'b-', alpha=0.6)
axes[0].set_ylabel('|Log Return|')
axes[0].set_title('Absolute Log Returns (Volatility Proxy)')

# Squared returns
axes[1].plot(df['Date'], df['log_return']**2, 'r-', alpha=0.6)
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Squared Log Return')
axes[1].set_title('Squared Log Returns')

plt.tight_layout()
plt.show()

In [ ]:
# ACF of squared returns (check for ARCH effects)
fig, ax = plt.subplots(figsize=(12, 4))
plot_acf((df['log_return']**2).dropna(), lags=50, ax=ax)
ax.set_title('ACF of Squared Log Returns (Evidence of Volatility Clustering)')
plt.tight_layout()
plt.show()

## 8. Event Impact Analysis (Visual)

In [ ]:
# Plot price around specific events
def plot_event_window(event_name, window_days=90):
    """Plot price series around a specific event."""
    event = events_df[events_df['event_name'] == event_name].iloc[0]
    event_date = event['date']
    
    # Get data window
    mask = (df['Date'] >= event_date - pd.Timedelta(days=window_days)) & \
           (df['Date'] <= event_date + pd.Timedelta(days=window_days))
    df_window = df[mask]
    
    # Plot
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.plot(df_window['Date'], df_window['Price'], 'b-', linewidth=1.5)
    ax.axvline(event_date, color='red', linestyle='--', linewidth=2, label=f'Event: {event_date.strftime("%Y-%m-%d")}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Price (USD/barrel)')
    ax.set_title(f'Price Around "{event_name}"\n{event["description"]}')
    ax.legend()
    plt.tight_layout()
    plt.show()
    
    # Calculate statistics
    before = df_window[df_window['Date'] < event_date]['Price'].mean()
    after = df_window[df_window['Date'] >= event_date]['Price'].mean()
    print(f"Average price {window_days} days before: ${before:.2f}")
    print(f"Average price {window_days} days after: ${after:.2f}")
    print(f"Change: {((after - before) / before) * 100:.2f}%")

In [ ]:
# Analyze specific events
plot_event_window('Global Financial Crisis Peak')

In [ ]:
plot_event_window('COVID-19 Pandemic')

In [ ]:
plot_event_window('Russia Ukraine Invasion')

## 9. Summary and Key Findings

### Data Overview
- **Period**: May 1987 to September 2022
- **Observations**: ~9,000 daily records
- **Price Range**: Approximately $10 to $140 USD/barrel

### Key Observations

1. **Non-stationarity**: Raw prices are non-stationary (confirmed by ADF and KPSS tests)
2. **Stationary Returns**: Log returns are stationary, making them suitable for modeling
3. **Volatility Clustering**: Clear evidence of volatility clustering (ARCH effects)
4. **Fat Tails**: Log returns exhibit excess kurtosis (leptokurtic distribution)
5. **Event Impact**: Visual inspection shows clear price reactions to major events

### Implications for Modeling

- Change point models should be applied to **log returns** for stationarity
- Multiple structural breaks are evident throughout the time series
- Volatility changes may be as important as mean changes
- Major events cluster around periods of high volatility

In [ ]:
# Save processed data for modeling
df.to_csv('../data/processed/brent_oil_processed.csv', index=False)
print("Processed data saved to ../data/processed/brent_oil_processed.csv")